In [94]:
import os
os.chdir('E:/IIM/TensorFlow/warranty/Bearings')

In [95]:
import numpy as np
a = np.load('E:/IIM/TensorFlow/warranty/Bearings/test_data.npy')
print(a[1])

[array([[151, 156, 158, ..., 146, 128, 109],
       [152, 157, 159, ..., 149, 131, 113],
       [152, 155, 159, ..., 152, 135, 119],
       ...,
       [174, 195, 215, ..., 255, 253, 240],
       [172, 198, 218, ..., 255, 253, 244],
       [175, 204, 225, ..., 255, 255, 246]], dtype=uint8)
 '10']


In [96]:
print("Size of the array: ", a[1].size)

Size of the array:  2


In [97]:
print("Length of one array element in bytes: ", a[1].itemsize)
print("Total bytes consumed by the elements of the array: ", a[1].nbytes)

Length of one array element in bytes:  8
Total bytes consumed by the elements of the array:  16


In [98]:
len(a[27])

2

In [99]:
len(a[:,1])  # there are 28 images in test_data npy file

28

In [100]:
# Classifying Cats vs Dogs with a Convolutional Neural Network on Kaggle3

# Below model from: https://pythonprogramming.net/convolutional-neural-network-kats-vs-dogs-machine-learning-tutorial/
# Import all libraries as below
# install OpenCV for first time

# Full run through of raw images to classification with Convolutional Neural Network : Dogs Vs Cats
# Package Requirements
#numpy (pip install numpy) tqdm (pip install tqdm)
#I will be using the GPU version of TensorFlow along with tflearn.

#To install the CPU version of TensorFlow, just do pip install tensorflow To install the GPU version of TensorFlow, 
# you need to get all the dependencies and such.

import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import os 
from random import shuffle
import cv2
%matplotlib inline

In [101]:
import os
cwd = os.getcwd()
cwd

'E:\\IIM\\TensorFlow\\warranty\\Bearings'

In [102]:
# Copy test and train datasets from Kaggle into the local PC and provide the pat of Test and Train Directories as below

#Once you have downloaded and extracted the data from https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data, 
# you're ready to begin.

# We've got the data, but we can't exactly just stuff raw images right through our convolutional neural network. 
# First, we need all of the images to be the same size, and then we also will probably want to just grayscale them. 
# Also, the labels of "gud" and "bad" are not useful, we want them to be one-hot arrays.

# TensorFlow Fold makes it easy to implement deep-learning models that operate over data of varying size and structure."

# Fascinating...but, for now, we'll do it the old fashioned way.

#TEST_DIR = "E:/IIM/TensorFlow/test"
#TRAIN_DIR = "E:/IIM/TensorFlow/train"

import os
os.chdir('E:/IIM/TensorFlow/warranty/Bearings')

TEST_DIR = "E:/IIM/TensorFlow/warranty/Bearings/test"
TRAIN_DIR = "E:/IIM/TensorFlow/warranty/Bearings/train"

In [103]:
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'gudVSbad-{}-{}.model'.format(LR, '2conv-basic')
# just so we remember which saved model is which, sizes must match
print(IMG_SIZE)

50


In [104]:
def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [gud,bad]
    #                            [much gud, no bad]
    if word_label == 'gud': return [1,0]
    #                             [no gud, very bad]
    elif word_label == 'bad': return [0,1]

In [105]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [106]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [107]:
#train_data = create_train_data()
# If you have already created the dataset:
train_data = np.load('train_data.npy')

In [108]:
#Next, we're ready to define our neural network:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression


In [109]:
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

In [110]:
# normalisation of images
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Create extra synthetic training data by flipping & rotating images
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [111]:
#convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input',data_preprocessing=img_prep, data_augmentation=img_aug)
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

In [112]:
import tensorflow as tf
tf.reset_default_graph()

In [113]:

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

In [114]:
#model = tflearn.DNN(convnet, tensorboard_dir='log')
#What we have here is a nice, 2 layered convolutional neural network, with a fully connected layer, 
# and then the output layer. It's been debated whether or not a fully connected layer is of any use. 
# I'll leave it in anyway. 

In [115]:
#This exact convnet was good enough for recognizing hand 28x28 written digits. 
# Let's see how it does with gud and bad bearings at 50x50 resolution.

In [116]:
# Now, it wont always be the case that you're training the network fresh every time. 
# Maybe first you just want to see how 3 epochs trains, but then, after 3, maybe you're done, 
# or maybe you want to see about 5 epochs. We want to be saving our model after every session, and reloading it
# if we have a saved version, so I will add this:

In [117]:
import os
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

In [118]:
#Now, let's split out training and testing data:

train = train_data[:-100]
test = train_data[-100:]

Now, the training data and testing data are both labeled datasets. The training data is what we'll fit the neural network with, and the test data is what we're going to use to validate the results. The test data will be "out of sample," meaning the testing data will only be used to test the accuracy of the network, not to train it.

We also have "test" images that we downloaded. THOSE images are not labeled at all, and those are what we'll submit to Kaggle for the competition.

Next, we're going to create our data arrays. For some reason, typical numpy logic like:

array[:,0] and array[:,1] did NOT work for me here. Not sure what I'm doing wrong, so I do this instead to separate my features and labels:

In [119]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

In [120]:
test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

In [121]:
# Now we fit for 3 epochs:

model.fit({'input': X}, {'targets': Y}, n_epoch=100, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=400, show_metric=True, run_id=MODEL_NAME)

Training Step: 599  | total loss: 13.97016 | time: 0.360s
| Adam | epoch: 100 | loss: 13.97016 - acc: 0.3933 -- iter: 320/378
Training Step: 600  | total loss: 13.79640 | time: 1.436s
| Adam | epoch: 100 | loss: 13.79640 - acc: 0.4008 | val_loss: 12.43396 - val_acc: 0.4600 -- iter: 378/378
--


In [ ]:
# Now we fit for 3 epochs:

#model.fit({'input': X}, {'targets': Y}, n_epoch=100, validation_set=({'input': test_x}, {'targets': test_y}), 
#    snapshot_step=400, show_metric=True, run_id=MODEL_NAME)
#Training Step: 599  | total loss: 9.40297 | time: 0.354s
#| Adam | epoch: 100 | loss: 9.40297 - acc: 0.5916 -- iter: 320/378
#Training Step: 600  | total loss: 9.29016 | time: 1.435s
#| Adam | epoch: 100 | loss: 9.29016 - acc: 0.5965 | val_loss: 10.59189 - val_acc: 0.5400 -- iter: 378/378
#--

In [ ]:
# It doesn't look like we've gotten anywhere at all.

#We could keep trying, but, if you haven't made accuracy progress in the first 3 epochs, you're probably not going to at all, 
#unless it's due to overfitment...at least in my experience.

#So... now what?
#  Size Matters¶
#We're gonna need a bigger network

#First, we need to reset the graph instance, since we're doing this in a continuous environment:

In [ ]:
import tensorflow as tf
tf.reset_default_graph()
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.5)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


In [125]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train = train_data[:-100]
test = train_data[-100:]

In [126]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

In [127]:
model.fit({'input': X}, {'targets': Y}, n_epoch=85, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 173  | total loss: 0.39037 | time: 0.402s
| Adam | epoch: 029 | loss: 0.39037 - acc: 0.8900 -- iter: 320/378
Training Step: 174  | total loss: 0.38503 | time: 1.477s
| Adam | epoch: 029 | loss: 0.38503 - acc: 0.8885 | val_loss: 0.21379 - val_acc: 0.9200 -- iter: 378/378
--


KeyboardInterrupt: 

Training Step: 497  | total loss: 0.20070 | time: 0.388s
| Adam | epoch: 083 | loss: 0.20070 - acc: 0.9579 -- iter: 320/378
Training Step: 498  | total loss: 0.18429 | time: 1.468s
| Adam | epoch: 083 | loss: 0.18429 - acc: 0.9621 | val_loss: 0.20660 - val_acc: 0.9400 -- iter: 378/378

####  WELL... Looks like we've got accuracy if 94%. With neural networks, size matters a ton. We went from having apparently 
#### un-trainable data to having obviously trainable data.

#If you are happy with the model, go ahead and save it:

In [ ]:
model.summary()

In [ ]:
# INFO:tensorflow:C:\Users\Eshwar\dogsvscats-0.001-2conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.

In [ ]:
#Now we can reload the model, and continue training (we don't NEED to reload the model here since this is continuous 
# and the model is still in memory, but if you were running this as a program you would)

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

In [ ]:
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.5)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')


In [ ]:
model = tflearn.DNN(convnet, tensorboard_dir='log')

In [ ]:
if os.path.exists('C:/Users/Eshwar/{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train = train_data[:-100]
test = train_data[-100:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch=70, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

In [ ]:
Training Step: 299  | total loss: 0.34085 | time: 0.405s
| Adam | epoch: 050 | loss: 0.34085 - acc: 0.9248 -- iter: 320/378
Training Step: 300  | total loss: 0.32763 | time: 1.481s
| Adam | epoch: 050 | loss: 0.32763 - acc: 0.9245 | val_loss: 0.20097 - val_acc: 0.9500 -- iter: 378/378

In [ ]:
# You can be too big.
#Bigger is not always better, there does get to be a limit, at least from my experience.
# A bigger network figures things out better, and quicker, but tends to also overfit the training data. 
# You can use dropout (sets randomly a certain % of nodes to not take part in the network for more robusts networks)
# to rectify this slightly, but there does seem to be a limit.

# Okay, now what? Let's see how we've done!

In [ ]:
# Visually inspecting our network against unlabeled data

In [ ]:
import matplotlib.pyplot as plt

# if you need to create the data:
test_data = process_test_data()
# if you already have some saved: Yes, saved in below fplder
#test_data = np.load('C:/User/Eshwar/test_data.npy')

fig=plt.figure()

for num,data in enumerate(test_data[:28]):
    # gud: [1,0]
    # bad: [0,1]
    
    img_num = data[1]
    img_data = data[0]
    
    y = fig.add_subplot(5,6,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    #model_out = model.predict([data])[0]
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: str_label='bad'
    else: str_label='gud'
        
    y.imshow(orig,cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()

Alright, so we made a couple of mistakes, but not too bad actually!

If you're happy with it, let's compete!

In [ ]:
with open('gudVsbadsubmission_file.csv','w') as f:
    f.write('id,label\n')
            
with open('gudVsbadsubmission_file.csv','a') as f:
    for data in tqdm(test_data):
        img_num = data[1]
        img_data = data[0]
        orig = img_data
        data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
        model_out = model.predict([data])[0]
        f.write('{},{}\n'.format(img_num,model_out[1]))

In [ ]:
#100%|██████████| 8/8 [00:00<00:00, 306.44it/s]

In [ ]:
#THE END

In [ ]:
import os
cwd = os.getcwd()
cwd